In [ ]:
import pandas as pd
data = pd.read_excel('/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx')
data.info()

In [6]:
df = data.drop(['سطح سوم' , 'تعداد'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18875 entries, 0 to 18874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   سوال    18875 non-null  object
 1   جواب    18874 non-null  object
dtypes: object(2)
memory usage: 295.1+ KB


In [15]:
df.head()

,سوال,جواب
0,در پردازش 717 (تبادل مانده انتقالی کاربر) پیغا...,در تمامی فعالیتهایی که منجر به ایجاد مانده انت...
1,جهت پیگیری مسائل مربوط به امور بازنشستگان بانک...,در صورتی که رمز ورود به سامانه فراموش شده است،...
2,بازخرید گواهی سپرده به صورت سیستمی انجام شده، ...,پردازش 911 (گزارش صورتحساب سی و پنج گردش آخر) ...
3,جهت فعال سازی اکسس کارت مشتری حقوقی از چه طریق...,ابتدا در پردازش 685 (فعال سازی کارت های مشتری ...
4,در پردازش 2937 (گزارش استعلام وضعیت درخواست دس...,ابتدا در پردازش 2934 (گزارش درخواست/تحویل دسته...


In [ ]:
import pandas as pd
import re

def delete_duplicate_sentences_in_excel(input_path: str, output_path: str, drop_column: list) -> None:
    
    # Load the workbook into a DataFrame
    df = pd.read_excel(input_path)
    df.drop(drop_column, axis=1, inplace=True)
    # Set to track sentences we've already kept
    seen_sentences = set()

    def dedup_text(text: str) -> str:
    
        if pd.isnull(text):
            return text
        # Split on punctuation followed by whitespace
        sentences = re.split(r'(?<=[.!?])\s+', str(text))
        result_parts = []
        for sentence in sentences:
            key = sentence.strip()
            if not key:
                continue
            if key not in seen_sentences:
                seen_sentences.add(key)
                result_parts.append(sentence)
            else:
                # duplicate — replace with blank
                result_parts.append('')
        # Rejoin, preserving order but skipping empty entries
        return ' '.join([s for s in result_parts if s]).strip()

    # Apply to every column in the DataFrame
    for column in df.columns:
        df[column] = df[column].apply(dedup_text)

    # Save the deduplicated DataFrame
    df.to_excel(output_path, index=False)
    print(f"Deduplication complete. Clean file saved to: {output_path}")
    # return df


if __name__ == "__main__":
    # Adjust these paths as needed
    input_file = "/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx"
    output_file = "/home/mahdi/word_embedding_Narenjestan/dataset/deduplicated_sentences.xlsx"
    delete_duplicate_sentences_in_excel(input_file, output_file, ['سطح سوم' , 'تعداد'])



In [ ]:
import pandas as pd
import re
from collections import Counter
from typing import List, Optional




 # --- Prepare splitter & cleaner ---
sentence_splitter = re.compile(r'(?<=[.!?])\s+')


# 1) Remove any leading "در پردازش <digits>"
REMOVE_PROCESS    = re.compile(r'\bپردازش\s*\d+\s*')

# 2) Inside your quoted error message, strip out
#    ". خطای شماره <digits> [مرحله <digits>]"
REMOVE_ERROR_TAIL = re.compile(r'\.\s*خطای شماره\s*\d+(?:\s*مرحله\s*\d+)?')

# 3) Remove any digits (and intervening spaces) immediately before "("
REMOVE_LEADING_NUM_PAREN = re.compile(r'\b\d+\s*\(')

def preprocess_text(text: str) -> str:
    # step 1
    text = REMOVE_PROCESS.sub('', text)
    # step 2
    text = REMOVE_ERROR_TAIL.sub('', text)
    # step 3
    # Turn "5912 (" or "5912   (" into "("
    text = REMOVE_LEADING_NUM_PAREN.sub('(', text)
    return text.strip()



def delete_sentences_by_repetition(
    input_path: str,
    output_path: str,
    drop_columns: Optional[List[str]] = None,
    repetition_threshold: int = 50
) -> None:
 
    drop_columns = drop_columns or []

    # --- 1) Load and drop ---
    df = pd.read_excel(input_path, dtype=str).fillna('')
    if drop_columns:
        df = df.drop(columns=drop_columns, errors='ignore')


    # --- 2) Count all (cleaned) sentences ---
    sentence_counts = Counter()
    for cell in df.values.flatten():

        cleaned_cell = preprocess_text(str(cell))
        for sent in sentence_splitter.split(cleaned_cell):
            s = sent.strip()
            if s:
                sentence_counts[s] += 1

    # --- 3) Build cleaned DataFrame ---
    def filter_cell(cell: str) -> str:
        kept = []
        cleaned_cell = preprocess_text(str(cell))
        for sent in sentence_splitter.split(cleaned_cell):
            s = sent.strip()
            # keep only if it exists and is under threshold
            if s and sentence_counts[s] < repetition_threshold:
                kept.append(s)
        return ' '.join(kept)

    cleaned_df = df.map(filter_cell)

    # --- 4) Write results ---
    counts_df = (
        pd.DataFrame(sentence_counts.items(), columns=['sentence', 'count'])
          .sort_values('count', ascending=False)
          .reset_index(drop=True)
    )

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        cleaned_df.to_excel(writer, sheet_name='Cleaned', index=False)
        counts_df.to_excel(writer, sheet_name='Sentence Counts', index=False)

    print(f"Done! Output saved to '{output_path}'")

# Example usage:
if __name__ == "__main__":
    delete_sentences_by_repetition(
        input_path="/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx",
        output_path="/home/mahdi/word_embedding_Narenjestan/dataset/V_3_deduplicated_sentences.xlsx",
        drop_columns=['سطح سوم' , 'تعداد'],
        repetition_threshold=20
    )


/tmp/ipykernel_2922623/2627332882.py:96: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = df.applymap(filter_cell)


Done! Output saved to '/home/mahdi/word_embedding_Narenjestan/dataset/V_3_deduplicated_sentences.xlsx'


In [9]:
sent = "در پردازش 5919 (گزارش وضعیت استعلام سیاح از بانک مرکزی) پاسخ استعلام با شرح'در انتظار پاسخ' نمایش داده می شود."

_REMOVE_PATTERN = re.compile(r'\b(?:در پردازش|پردازش)\s*\d+\b')

_REMOVE_PATTERN.sub('', sent).strip()

"(گزارش وضعیت استعلام سیاح از بانک مرکزی) پاسخ استعلام با شرح'در انتظار پاسخ' نمایش داده می شود."

In [12]:
# 1) Remove any leading "در پردازش <digits>"
REMOVE_PROCESS    = re.compile(r'\bدر پردازش\s*\d+\s*')

# 2) Inside your quoted error message, strip out
#    ". خطای شماره <digits> [مرحله <digits>]"
REMOVE_ERROR_TAIL = re.compile(r'\.\s*خطای شماره\s*\d+(?:\s*مرحله\s*\d+)?')

# 3) Remove any digits (and intervening spaces) immediately before "("
REMOVE_LEADING_NUM_PAREN = re.compile(r'\b\d+\s*\(')

def clean_sentence(text: str) -> str:
    # step 1
    text = REMOVE_PROCESS.sub('در ', text)
    # step 2
    text = REMOVE_ERROR_TAIL.sub('', text)
    # step 3
    # Turn "5912 (" or "5912   (" into "("
    text = REMOVE_LEADING_NUM_PAREN.sub('(', text)
    return text.strip()

orig = (
    'خطای شماره 403875" دریافت می گردد.'
)

print(clean_sentence(orig))

sent = 'در (ابطال چک تخصیص یافته) پیغام خطای "تمام سریال های وارد شده شرایط ابطال ندارند. خطای شماره 3504 مرحله 6" دریافت می گردد.'
print(clean_sentence(sent))


sent2= 'در پردازش 4612 (ابطال ضمانتنامه) پیغام خطای "اجرای عملیات ابطال ملزم به استفاده از پردازش 4489 می باشد. خطای شماره 3184744" دریافت می گردد.'

print(clean_sentence(sent2))

خطای شماره 403875" دریافت می گردد.
در (ابطال چک تخصیص یافته) پیغام خطای "تمام سریال های وارد شده شرایط ابطال ندارند" دریافت می گردد.
در (ابطال ضمانتنامه) پیغام خطای "اجرای عملیات ابطال ملزم به استفاده از پردازش 4489 می باشد" دریافت می گردد.


In [20]:
import pandas as pd
from preprocessing_main import preprocess

input_path = "/home/mahdi/word_embedding_Narenjestan/dataset/MEC-Narenjestan_cleaned-V0.2.xlsx"
df_edited = pd.read_excel(input_path, dtype=str).fillna('')

# Clean every cell in the DataFrame
df_clean = df_edited.map(
        lambda cell: preprocess(
        str(cell),
        remove_punctuations=True,
        replace_multiple_spaces=True
    )
)

df_clean.to_excel("/home/mahdi/word_embedding_Narenjestan/dataset/MEC-Narenjestan_cleaned-V0.3.xlsx",index=False)
# Now you can continue with splitting, counting, etc.


In [18]:
df_clean.head()

,سوال,جواب
0,در تبادل مانده انتقالی کاربر پیغام خطای کد ملی...,در تمامی فعالیتهایی که منجر به ایجاد مانده انت...
1,جهت پیگیری مسائل مربوط به امور بازنشستگان بانک...,در صورتی که رمز ورود به سامانه فراموش شده است ...
2,بازخرید گواهی سپرده به صورت سیستمی انجام شده ا...,گزارش صورتحساب سی و پنج گردش آخر بررسی گردد در...
3,جهت فعال سازی اکسس کارت مشتری حقوقی از چه طریق...,ابتدا در فعال سازی کارت های مشتری حقوقی شماره ...
4,در گزارش استعلام وضعیت درخواست دسته چک پیغام خ...,ابتدا در گزارش درخواست تحویل دسته چک در کلیه ش...


In [23]:
import pandas as pd
import re
from collections import defaultdict

def dedup_ngrams_in_excel(input_path: str,
                          output_path: str,
                          max_occurrences: int = 10):
    
    # 1) Load
    df = pd.read_excel(input_path)

    # 2) For each n-gram length
    for n in [6, 5, 4, 3, 2]:
        counts = defaultdict(int)

        def dedup_cell(text):
            if pd.isnull(text):
                return text
            # simple tokenization (words and punctuation split)
            tokens = re.findall(r"\w+|[^\w\s]", str(text), re.UNICODE)
            result = list(tokens)  # copy for in-place blanking

            # slide window
            for i in range(len(tokens) - n + 1):
                gram = tuple(tokens[i:i+n])
                counts[gram] += 1
                if counts[gram] > max_occurrences:
                    # blank out this over-quota n-gram
                    for j in range(i, i+n):
                        result[j] = ""

            # reconstruct: drop blanks, join with spaces
            cleaned = " ".join(tok for tok in result if tok)
            return cleaned

        # apply to every cell
        df = df.applymap(dedup_cell)

    # 3) Save
    df.to_excel(output_path, index=False)
    print(f"Done – cleaned file saved to '{output_path}'.")



In [ ]:

if __name__ == "__main__":
    in_file = "/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx"              
    out_file = "/home/mahdi/word_embedding_Narenjestan/dataset/dedup_ngrams_output.xlsx"        
    dedup_ngrams_in_excel(in_file, out_file)


/tmp/ipykernel_1489679/255363609.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(dedup_cell)


Done – cleaned file saved to '/home/mahdi/word_embedding_Narenjestan/dataset/dedup_ngrams_output.xlsx'.


## NGRAM with filter

In [ ]:
import pandas as pd
import re
from collections import Counter

def extract_and_save_ngrams(input_xlsx: str,
                            output_xlsx: str,
                            min_n: int = 2,
                            max_n: int = 6,
                            drop_column: list = None,
                            valid_count: int = 10):
   
    drop_column = drop_column or []
    # 1) Load your data
    df = pd.read_excel(input_xlsx, dtype=str).fillna('')
    if drop_column:
        df = df.drop(columns=drop_column, errors='ignore')

    # 2) Concatenate all cells into one big lowercase string
    full_text = df.astype(str).agg(' '.join, axis=1).str.cat(sep=' ').lower()

    # 3) Tokenize on word characters
    tokens = re.findall(r'\w+', full_text)

    # Helper: sliding-window generator
    def ngrams(tokens, n):
        return zip(*(tokens[i:] for i in range(n)))

    # 4) Prepare an Excel writer
    with pd.ExcelWriter(output_xlsx, engine='openpyxl') as writer:
        for n in range(min_n, max_n + 1):
            # Count all n-grams
            counts = Counter(ngrams(tokens, n))
            # Filter to only those >= valid_count
            filtered = {gram: cnt for gram, cnt in counts.items() if cnt >= valid_count}
            if not filtered:
                # nothing to write for this n
                continue

            # Build a DataFrame from filtered counts
            df_ng = (
                pd.DataFrame(filtered.items(), columns=['ngram_tuple', 'count'])
                  .assign(ngram=lambda d: d['ngram_tuple']
                                         .apply(lambda tup: ' '.join(tup)))
                  .loc[:, ['ngram', 'count']]
                  .sort_values('count', ascending=False)
                  .reset_index(drop=True)
            )

            # Write to its own sheet
            sheet_name = f"{n}-grams"
            df_ng.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"All done! N-grams (count ≥ {valid_count}) saved to '{output_xlsx}'.")


if __name__ == "__main__":
    extract_and_save_ngrams(
        input_xlsx="/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx" ,        # ← your source file
        output_xlsx="ngram_counts.xlsx",    # ← where to write results
        min_n=2,
        max_n=10,
        drop_column=['سطح سوم' , 'تعداد']
    )

## NGRAM with filter (using NLTK for tokenization)

In [ ]:
# # using nltk

# import pandas as pd
# import nltk
# from nltk import ngrams
# from nltk.tokenize import wordpunct_tokenize
# from collections import Counter

# def extract_and_save_ngrams_nltk(input_xlsx: str,
#                                  output_xlsx: str,
#                                  min_n: int = 2,
#                                  max_n: int = 6,
#                                  drop_column: list = None,
#                                  valid_count: int = 10):

#     drop_column = drop_column or []

#     # 1) Load data
#     df = pd.read_excel(input_xlsx, dtype=str).fillna('')
#     if drop_column:
#         df = df.drop(columns=drop_column, errors='ignore')

#     # 2) Combine all text into one lowercase blob
#     full_text = df.astype(str).agg(' '.join, axis=1).str.cat(sep=' ').lower()

#     # 3) Tokenize into words and punctuation (no external models)
#     tokens = wordpunct_tokenize(full_text)

#     # 4) Prepare Excel writer
#     with pd.ExcelWriter(output_xlsx, engine='openpyxl') as writer:
#         for n in range(min_n, max_n + 1):
#             # Count all n-grams
#             cnt = Counter(ngrams(tokens, n))
#             # Keep only those with count >= valid_count
#             filtered = {gram: c for gram, c in cnt.items() if c >= valid_count}
#             if not filtered:
#                 continue

#             # Build DataFrame
#             df_ng = (
#                 pd.DataFrame(filtered.items(), columns=['ngram_tuple', 'count'])
#                   .assign(
#                       ngram=lambda d: d['ngram_tuple']
#                                          .apply(lambda tup: ' '.join(tup))
#                   )
#                   .loc[:, ['ngram', 'count']]
#                   .sort_values('count', ascending=False)
#                   .reset_index(drop=True)
#             )

#             # Write to sheet named "2-grams", "3-grams", etc.
#             sheet = f"{n}-grams"
#             df_ng.to_excel(writer, sheet_name=sheet, index=False)

#     print(f"Done! N-grams (count ≥ {valid_count}) saved to '{output_xlsx}'.")

# if __name__ == "__main__":
#     extract_and_save_ngrams_nltk(
#         input_xlsx="/home/mahdi/word_embedding_Narenjestan/dataset/narenjestan_khowledgebase_editable.xlsx" ,        # ← your source file
#         output_xlsx="ngram_counts_nltk.xlsx",    # ← where to write results
#         min_n=2,
#         max_n=10,
#         drop_column=['سطح سوم' , 'تعداد']
#     )